In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
# https://www.youtube.com/watch?v=ny215UUXbhI&list=PLOrU905yPYXLwua9Ci5rXKA8iRz61bpg7&index=1

In [3]:
def function_1(input_1):
    return input_1 + " first function"


def function_2(input_2):
    return input_2 + " second function"

In [4]:
from langgraph.graph import Graph

workflow = Graph()

workflow.add_node("node_1", function_1)
workflow.add_node("node_2", function_2)

workflow.add_edge("node_1", "node_2")

workflow.set_entry_point("node_1")
workflow.set_finish_point("node_2")

app = workflow.compile()

In [5]:
app.invoke("Hello, world!")

'Hello, world! first function second function'

In [6]:
input  = "I am moving from"

for output in app.stream(input):
    for k, v in output.items():
        print(f"Name of node: {k}")
        print("values: ", v)

    print("\n\n")

Name of node: node_1
values:  I am moving from first function



Name of node: node_2
values:  I am moving from first function second function





In [13]:
from langchain_openai import ChatOpenAI
from langchain_community.document_loaders import TextLoader
from langchain_mistralai.chat_models import ChatMistralAI
from IPython.display import Image

In [11]:
# If api_key is not passed, default behavior is to use the `MISTRAL_API_KEY` environment variable.
llm = ChatOpenAI()

In [12]:
llm.invoke("How much data do you need to train a model?")

AIMessage(content='The amount of data needed to train a model can vary depending on the complexity of the task, the size of the model, and the quality of the data. In general, more data is better for training a model as it allows for better generalization and performance. However, there is no fixed amount of data that is considered sufficient for training a model. It is recommended to have at least thousands of data points for simple tasks, and tens of thousands to millions of data points for more complex tasks like deep learning models. It is also important to have a diverse and representative dataset to ensure that the model can generalize well to unseen data.', response_metadata={'token_usage': {'completion_tokens': 127, 'prompt_tokens': 18, 'total_tokens': 145}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-af72fefd-29da-4a37-80df-4e42dbc9e450-0', usage_metadata={'input_tokens': 18, 'output_tokens': 127, 'total_t

In [15]:
# New

In [14]:
def function_1(input_1):
    complete_query = "Your task is to provide only the topic based on the user query" \
    "Only output the topic amont: [Japan, Sports]. Don't include reasoning. Following is " \
    "the user query: " + input_1

    response = llm.invoke(complete_query)
    return response.content


def function_2(input_2):
    TOPIC_UPPER = input_2.upper()
    response = f"Here is the topic in UPPER case: {TOPIC_UPPER}"
    return response

In [16]:
from langgraph.graph import Graph

workflow = Graph()

workflow.add_node("agent", function_1)
workflow.add_node("tool", function_2)

workflow.add_edge("agent", "tool")

workflow.set_entry_point("agent")
workflow.set_finish_point("tool")

app = workflow.compile()

In [17]:
query = "Tell me about Japan's early history"
app.invoke(query)

'Here is the topic in UPPER case: JAPAN'

In [18]:
# 3 with agent state

In [19]:
AgentState = dict()

AgentState["messages"] = list()

In [67]:
from tavily import TavilyClient
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.tools import tool
from langchain_core.output_parsers import StrOutputParser


@tool
def search(query: str) -> str:
    """Searches the internet for something about the query."""
    tavily_client = TavilyClient()
    r = tavily_client.search(query, max_results=1)
    answer = r.get("results")[0].get("content")
    return {"context": answer, "question": query}


# r = search("Japan's early history")
# print(r)
# print(r.get("results")[0].get("content"))


In [68]:
def function_1(state):
    messages = state["messages"]
    question = messages[-1]

    complete_query = "Your task is to provide only the topic based on the user query" \
    "Only output the topic amont: [Japan, Sports]. Don't include reasoning. Following is " \
    "the user query: " + question

    response = llm.invoke(complete_query)
    state["messages"].append(response.content)
    return state


def function_2(state):
    messages = state["messages"]
    question = messages[0]

    template = """Create 3 bullet points answering the question based on on the following context:
    {context}

    question: {question}    
    """
    prompt = ChatPromptTemplate.from_template(template)
    retrieval_chain = (
        search
        | prompt
        | llm
        | StrOutputParser()
    )
    result = retrieval_chain.invoke(question)
    return result




In [69]:
workflow = Graph()

workflow.add_node("agent", function_1)
workflow.add_node("tool", function_2)

workflow.add_edge("agent", "tool")

workflow.set_entry_point("agent")
workflow.set_finish_point("tool")

app = workflow.compile()

input = {"messages": ["Tell me about Japan's early history"]}

# app.invoke(input)


for output in app.stream(input):
    for k, v in output.items():
        print(f"Name of node: {k}")
        print("Answer:")
        print(v)

    print("\n\n")

Name of node: agent
Answer:
{'messages': ["Tell me about Japan's early history", 'Japan']}



Name of node: tool
Answer:
- Japan's early history is marked by periods such as the Jomon, Yayoi, Kofun, and Asuka periods, each contributing to the cultural development of the nation.
- The influence of these early periods can still be seen in traditional arts, architecture, religious practices, and societal norms in Japan today.
- Studying Japan's early history can provide insights into the foundation of the nation's identity and help us understand how certain aspects of Japanese culture have evolved over time.





In [70]:
from typing import TypedDict, Annotated, Sequence
import operator
from langchain_core.messages import BaseMessage


class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]

In [71]:
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
from langchain.prompts import PromptTemplate


class TopicSelectionParser(BaseModel):
    Topic: str = Field(description="Selected Topic")


parser = PydanticOutputParser(pydantic_object=TopicSelectionParser)

In [142]:
@tool
def search(query: str) -> str:
    """Searches the internet for something about the query."""
    tavily_client = TavilyClient()
    r = tavily_client.search(query, max_results=1)
    answer = r.get("results")[0].get("content")
    return {"context": answer, "question": query}
    # return answer


def function_1(state):
    print("--> calling agent...")
    messages = state["messages"]
    question = messages[-1]

    template = """Your task is to provide only the topic based on the user query 
    Only output the topic amont: [Japan, Sports, Not Relevant]. Don't include reasoning. Following is the user query: {question}\n
    {format_instructions}"""

    prompt = PromptTemplate.from_template(
        template,
        # input_variables=["question"],
        # partial_variables={"format_instructions": parser.get_format_instructions()},
    )
    prompt.format(question=question, format_instructions=parser.get_format_instructions())

    chain = prompt | llm | parser
    response = chain.invoke({
        "question": question,
        "format_instructions": parser.get_format_instructions(),
    })
    print(response.Topic)
    return {"messages": [response.Topic]}


def function_2(state):
    print("--> calling search tool and make bullets...")
    messages = state["messages"]
    question = messages[0]

    template = """Create 3 bullet points answering the question based on on the following context:
    {context}

    question: {question}    
    """
    prompt = ChatPromptTemplate.from_template(template)

    retrieval_chain = (
        search
        | prompt
        | llm
        | StrOutputParser()
    )
    result = retrieval_chain.invoke(question)
    return {"messages": [result]}


def function_3(state):
    print("--> calling search tool and make long text...")
    messages = state["messages"]
    question = messages[0]

    template = """Create a short sentence answering the question based on on the following context:
    {context}

    question: {question}    
    """
    prompt = ChatPromptTemplate.from_template(template)
    retrieval_chain = (
        search
        | prompt
        | llm
        | StrOutputParser()
    )
    result = retrieval_chain.invoke(question)
    return {"messages": [result]}


def router(state):
    print("--> routing...")
    messages = state["messages"]
    last_message = messages[-1]

    print(last_message)
    if "Japan" in last_message or "Sports" in last_message:
        return "bullet_points"
    else:
        return "short_text"


In [143]:
from langgraph.graph import StateGraph, END

graph = StateGraph(AgentState)

graph.add_node("agent", function_1)
graph.add_node("tool_with_bullets", function_2)
graph.add_node("tool_with_short_text", function_3)
graph.set_entry_point("agent")

graph.add_conditional_edges(
    source="agent",
    path=router,
    path_map={
        "bullet_points": "tool_with_bullets",
        "short_text": "tool_with_short_text",
    },
)

graph.add_edge("tool_with_bullets", END)
graph.add_edge("tool_with_short_text", END)

app = graph.compile()

In [144]:
input = {"messages": ["Tell me about Japan's early history"]}

for output in app.stream(input):
    for k, v in output.items():
        print(f"Name of node: {k}")
        print("Answer:")
        print(v.get("messages")[0])

    print("\n\n")

--> calling agent...
Japan
--> routing...
Japan
Name of node: agent
Answer:
Japan



--> calling search tool and make bullets...
Name of node: tool_with_bullets
Answer:
- The Jomon Period is considered the first historical period of Japan, spanning from around 14,500 to 300 BCE.
- The name "Jomon" comes from the distinctive pottery produced during this time, which featured simple rope-like decoration.
- The Jomon Period is known for its hunter-gatherer lifestyle and early agricultural practices, laying the foundation for Japan's future development.





In [145]:
input = {"messages": ["Tell me a bit about Denmark"]}

for output in app.stream(input):
    for k, v in output.items():
        print(f"Name of node: {k}")
        print("Answer:")
        print(v.get("messages")[0])

    print("\n\n")

--> calling agent...
Not Relevant
--> routing...
Not Relevant
Name of node: agent
Answer:
Not Relevant



--> calling search tool and make long text...
Name of node: tool_with_short_text
Answer:
Denmark is a Nordic country with the oldest state flag in the world still in use by an independent nation.





## Tool Calling

In [146]:
# https://www.youtube.com/watch?v=UVqDDM5CWYQ&list=PLOrU905yPYXLwua9Ci5rXKA8iRz61bpg7&index=2

In [147]:
model = ChatOpenAI(temperature=0.5)

In [148]:
model.invoke("Hi")

AIMessage(content='Hello! How can I assist you today?', response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 8, 'total_tokens': 17}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-b14ab4fb-426d-49ba-941b-0ab3b6e99cbf-0', usage_metadata={'input_tokens': 8, 'output_tokens': 9, 'total_tokens': 17})

In [152]:
import json
from langchain_core.messages import ToolMessage
from langchain_core.tools import tool
from langchain_core.utils.function_calling import convert_to_openai_tool
from pprint import pprint

@tool
def multiply(a: int, b: int) -> int:
    """Mulitplies two numbers and returns the result."""
    return a * b


model_with_tools = model.bind(tools=[convert_to_openai_tool(multiply)])

In [150]:
r = model_with_tools.invoke("What is 44 * 11?")
print(r)

content='' additional_kwargs={'tool_calls': [{'id': 'call_tZNxvA7W962gJsJyhf4VJD3Q', 'function': {'arguments': '{"a":44,"b":11}', 'name': 'multiply'}, 'type': 'function'}]} response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 59, 'total_tokens': 76}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None} id='run-01dda886-d8b7-40c7-bf37-7faecccfe817-0' tool_calls=[{'name': 'multiply', 'args': {'a': 44, 'b': 11}, 'id': 'call_tZNxvA7W962gJsJyhf4VJD3Q', 'type': 'tool_call'}] usage_metadata={'input_tokens': 59, 'output_tokens': 17, 'total_tokens': 76}


In [153]:
tool_calls = r.additional_kwargs.get("tool_calls")
pprint(tool_calls)

[{'function': {'arguments': '{"a":44,"b":11}', 'name': 'multiply'},
  'id': 'call_tZNxvA7W962gJsJyhf4VJD3Q',
  'type': 'function'}]


In [156]:
for tool_call in tool_calls:
    print(f"Function name: {tool_call.get("function").get("name")}")
    print(f"Function arguments: {tool_call.get("function").get("arguments")}")
    print(tool_call)

Function name: multiply
Function arguments: {"a":44,"b":11}
{'id': 'call_tZNxvA7W962gJsJyhf4VJD3Q', 'function': {'arguments': '{"a":44,"b":11}', 'name': 'multiply'}, 'type': 'function'}


In [157]:
from typing import TypedDict, Annotated, Sequence
import operator
from langchain_core.messages import BaseMessage


class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]

In [160]:
from langgraph.graph import StateGraph, END

graph = StateGraph(AgentState)


def invoke_model(state):
    print("--> calling model...")
    messages = state["messages"]
    question = messages[-1]
    return {"messages": [model_with_tools.invoke(question)]}


def invoke_tool(state):
    print("--> calling tool...")
    tool_calls = state["messages"][-1].additional_kwargs.get("tool_calls", [])
    multiply_call = None

    for tool_call in tool_calls:
        tool_name = tool_call.get("function").get("name")
        if tool_name == "multiply":
            multiply_call = tool_call

    if multiply_call is None:
        raise Exception("No multiply call found.")
    
    r = multiply.invoke(json.loads(multiply_call.get("function").get("arguments")))
    return {"messages": [r]}


def router(state):
    print("--> routing...")
    tool_calls = state["messages"][-1].additional_kwargs.get("tool_calls", [])
    if len(tool_calls) > 0:
        return "multiply"
    else:
        return "end"
    


graph.add_node("agent", invoke_model)
graph.add_node("tool", invoke_tool)
graph.add_edge("tool", END)
graph.set_entry_point("agent")
graph.add_conditional_edges(
    "agent",
    router,
    {
        "multiply": "tool",
        "end": END,
    },
)

app = graph.compile()

output = app.invoke({"messages": ["What is 44 * 11?"]})
print(output)

--> calling model...
--> routing...
--> calling tool...
{'messages': ['What is 44 * 11?', AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_xFqggSMuzvCjQfBrikCqUT3e', 'function': {'arguments': '{"a":44,"b":11}', 'name': 'multiply'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 59, 'total_tokens': 76}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-99b8db48-e879-4a2d-ab42-f277c1b64a9e-0', tool_calls=[{'name': 'multiply', 'args': {'a': 44, 'b': 11}, 'id': 'call_xFqggSMuzvCjQfBrikCqUT3e', 'type': 'tool_call'}], usage_metadata={'input_tokens': 59, 'output_tokens': 17, 'total_tokens': 76}), 484]}


In [161]:
output = app.invoke({"messages": ["How do you describe a honey bee?"]})
print(output)

--> calling model...
--> routing...
{'messages': ['How do you describe a honey bee?', AIMessage(content='A honey bee is a small flying insect known for its role in pollination and honey production. Honey bees are social insects that live in colonies with a queen, worker bees, and drones. They have a yellow and black striped body, wings, and a stinger. Honey bees are essential for the pollination of plants and crops, making them crucial for agriculture and ecosystem health.', response_metadata={'token_usage': {'completion_tokens': 77, 'prompt_tokens': 59, 'total_tokens': 136}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-9e9a07fb-22f4-4d19-8039-ff49e5b00559-0', usage_metadata={'input_tokens': 59, 'output_tokens': 77, 'total_tokens': 136})]}


# Multi Agent Supervisor

In [1]:
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain_core.messages import BaseMessage, HumanMessage
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

llm = ChatOpenAI()

In [2]:
# Agent Function
def create_agent(llm: ChatOpenAI, tools: list, system_prompt: str):
    prompt = ChatPromptTemplate.from_messages(
        [
            ("system", system_prompt),
            MessagesPlaceholder(variable_name="messages"),
            MessagesPlaceholder(variable_name="messages"),

        ]
    )
    agent = create_openai_tools_agent(llm, tools, prompt)
    executor = AgentExecutor(agent=agent, tools=tools)
    return executor

In [4]:
# Create tools
from typing import TypedDict, Annotated, Sequence, List, Tuple, Union
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.tools import tool
from langchain_experimental.tools import PythonREPLTool
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough


tavily_tool = TavilySearchResults(max_results=2)
python_repl_tool = PythonREPLTool()

In [5]:
def agent_node(state, agent, name):
    result = agent.invoke(state)
    return {"messages": [HumanMessage(content=result["output"], name=name)]}

In [8]:
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

In [9]:
members = ["RAG", "Researcher", "Coder"]
system_prompt = (
    "You are a supervisor tasked with managing a conversation between the "
    "following members: {members}. Given the following user request, "
    "respond with the worker to act next. Use RAG tool when questions are "
    "related to Japan. Each worker wil perform a task and respond with their "
    "results and status. When finished, respond with FINISH."
)

options = ["FINISH"] + members

function_def = {
    "name": "route",
    "description": "Select the next role.",
    "parameters": {
        "title": "routeSchema",
        "type": "object",
        "properties": {
            "next": {
                "title": "Next",
                "anyOf": [
                    {"enum": options},
                ]
            }
        },
        "required": ["next"],
    }
}
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder(variable_name="messages"),
        (
            "system",
            "Given the conversation above, who should act next? or should we FINISH? Select one of: {options}",
        ),
    ]
).partial(options=str(options), members=", ".join(members))

supervisor_chain = (
    prompt
    | llm.bind_functions(functions=[function_def], function_call="route")
    | JsonOutputFunctionsParser()
)

In [10]:
import operator
from typing import Annotated, Any, Dict, List, Optional, Sequence, TypedDict
import functools

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langgraph.graph import StateGraph, END

In [ ]:
class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]
    next: str


research_agent = create_agent(llm, [tavily_tool], "You are a web researcher")
research_node = functools.partial(agent_node, agent=research_agent, name="Researcher")

code_agent = create_agent(
    llm,
    [python_repl_tool],
    "You are a coder. You will write code to solve the problem."
)
code_node = functools.partial(agent_node, agent=code_agent, name="Coder")
